In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e12/sample_submission.csv
/kaggle/input/playground-series-s4e12/train.csv
/kaggle/input/playground-series-s4e12/test.csv


# library gpus

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.model_selection import ParameterSampler, KFold

import warnings
warnings.filterwarnings("ignore")

import torch

def check_gpu_status():
    if torch.cuda.is_available():
        print(f"可用GPU数量: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
            print(f"显存使用情况: {torch.cuda.memory_allocated(i) / 1024**2:.2f} MB 已用, {torch.cuda.memory_reserved(i) / 1024**2:.2f} MB 总分配")
            print(f"显存空余: {torch.cuda.memory_reserved(i) - torch.cuda.memory_allocated(i):.2f} 字节\n")
    else:
        print("未检测到 GPU！")

check_gpu_status()



可用GPU数量: 2
GPU 0: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节

GPU 1: Tesla T4
显存使用情况: 0.00 MB 已用, 0.00 MB 总分配
显存空余: 0.00 字节



# IMPORT DATA

In [3]:

train = pd.read_csv("/kaggle/input/playground-series-s4e12/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv")

sample = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True) 
 



# data prosessing

In [4]:
def date(Df):

    Df['Policy Start Date'] = pd.to_datetime(Df['Policy Start Date'])
    Df['Year'] = Df['Policy Start Date'].dt.year
    Df['Day'] = Df['Policy Start Date'].dt.day
    Df['Month'] = Df['Policy Start Date'].dt.month
    Df['Month_name'] = Df['Policy Start Date'].dt.month_name()
    Df['Day_of_week'] = Df['Policy Start Date'].dt.day_name()
    Df['Week'] = Df['Policy Start Date'].dt.isocalendar().week
    Df['Year_sin'] = np.sin(2 * np.pi * Df['Year'])
    Df['Year_cos'] = np.cos(2 * np.pi * Df['Year'])
    Df['Month_sin'] = np.sin(2 * np.pi * Df['Month'] / 12) 
    Df['Month_cos'] = np.cos(2 * np.pi * Df['Month'] / 12)
    Df['Day_sin'] = np.sin(2 * np.pi * Df['Day'] / 31)  
    Df['Day_cos'] = np.cos(2 * np.pi * Df['Day'] / 31)
    Df['Group']=(Df['Year']-2020)*48+Df['Month']*4+Df['Day']//7
    
    Df.drop('Policy Start Date', axis=1, inplace=True)

    return Df
train = date(train)
test = date(test)

cat_cols = [col for col in train.columns if train[col].dtype == 'object']
feature_cols = list(test.columns)
class CategoricalEncoder:
    def __init__(self, train, test):
        self.train = train
        self.test = test

    def frequency_encode(self, cat_cols, feature_cols, drop_org=False):
        combined = pd.concat([self.train, self.test], axis=0, ignore_index=True)

        new_cat_cols = [] 
        for col in cat_cols:
            freq_encoding = combined[col].value_counts().to_dict()
            
            self.train[f"{col}_freq"] = self.train[col].map(freq_encoding).astype('float')
            self.test[f"{col}_freq"] = self.test[col].map(freq_encoding).astype('float')

            new_col_name = f"{col}_freq"
            new_cat_cols.append(new_col_name)
            feature_cols.append(new_col_name)
            if drop_org:
                feature_cols.remove(col)

        return self.train, self.test, new_cat_cols, feature_cols
encoder = CategoricalEncoder(train, test)
train, test, cat_cols, feature_cols = encoder.frequency_encode(cat_cols, feature_cols, drop_org=True)

train = train[feature_cols + ['Premium Amount']]
test = test[feature_cols]

In [5]:
X = train.drop('Premium Amount', axis=1)  
y = train['Premium Amount']

y_log = np.log1p(y)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# xgboost

In [6]:
# 定义 XGBoost 的随机搜索训练函数
def random_search_train_xgb(X, y, param_list, n_splits=5, random_state=42):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    best_score = float('inf')
    best_params = None
    best_models = None
    results = []

    for idx, params in enumerate(param_list):
        print(f"Search {idx+1}/{len(param_list)} with params: {params}")
        oof = np.zeros(len(X))
        models = []
        fold_scores = []

        for fold, (train_idx, valid_idx) in enumerate(kf.split(X)):
            print(f"  Fold {fold + 1}")
            X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            y_train, y_valid = np.log1p(y.iloc[train_idx]), np.log1p(y.iloc[valid_idx])

            model = xgb.XGBRegressor(
                **params,
                objective='reg:squarederror',
                random_state=random_state,
                eval_metric="rmse",
                verbosity=0,
                tree_method='gpu_hist',  # 使用 GPU 加速
                booster='gbtree',
            )
            
            model.fit(
                X_train, y_train,
                eval_set=[(X_valid, y_valid)],
                early_stopping_rounds=300,
                verbose=False,
            )
            models.append(model)

            # 预测和评估
            oof[valid_idx] = np.maximum(0, model.predict(X_valid))
            fold_rmsle = rmsle(np.expm1(y_valid), np.expm1(oof[valid_idx]))
            fold_scores.append(fold_rmsle)

            print(f"    Fold {fold + 1} RMSLE: {fold_rmsle:.4f}")
        
        mean_score = np.mean(fold_scores)
        print(f"Mean RMSLE for Search {idx+1}: {mean_score:.4f}")

        # 保存最佳模型
        if mean_score < best_score:
            best_score = mean_score
            best_params = params
            best_models = models

        # 记录结果
        results.append({'params': params, 'mean_rmsle': mean_score})
    
    results_df = pd.DataFrame(results)
    print("\nBest Parameters:", best_params)
    print(f"Best RMSLE: {best_score:.4f}")
    return best_models, best_params, results_df

# 定义超参数搜索空间
param_grid = {
    'n_estimators': [1000, 2000, 3000],
    'learning_rate': [0.01, 0.03, 0.05],
    'max_depth': [4, 6, 8, 10],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.3],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [0, 0.1, 1.0]
}

# 随机抽取参数
n_searches = 15
param_list = list(ParameterSampler(param_grid, n_iter=n_searches, random_state=42))

# 执行随机搜索训练
models, best_params, results_df = random_search_train_xgb(X, y, param_list)

# 对测试集进行预测
test_predictions = np.zeros(len(test))
for model in models:
    test_predictions += np.maximum(0, np.expm1(model.predict(test))) / len(models)

# 将预测结果保存到 CSV 文件
sample['Premium Amount'] = test_predictions
sample.to_csv('submission_xgb.csv', index=False)


Search 1/15 with params: {'subsample': 1.0, 'reg_lambda': 0, 'reg_alpha': 0, 'n_estimators': 1000, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 1.0}
  Fold 1
    Fold 1 RMSLE: 1.0468
  Fold 2
    Fold 2 RMSLE: 1.0458
  Fold 3
    Fold 3 RMSLE: 1.0469
  Fold 4
    Fold 4 RMSLE: 1.0449
  Fold 5
    Fold 5 RMSLE: 1.0464
Mean RMSLE for Search 1: 1.0461
Search 2/15 with params: {'subsample': 0.7, 'reg_lambda': 0, 'reg_alpha': 0, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 6, 'learning_rate': 0.03, 'gamma': 0.3, 'colsample_bytree': 0.8}
  Fold 1
    Fold 1 RMSLE: 1.0468
  Fold 2
    Fold 2 RMSLE: 1.0458
  Fold 3
    Fold 3 RMSLE: 1.0468
  Fold 4
    Fold 4 RMSLE: 1.0448
  Fold 5
    Fold 5 RMSLE: 1.0463
Mean RMSLE for Search 2: 1.0461
Search 3/15 with params: {'subsample': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 0.1, 'n_estimators': 2000, 'min_child_weight': 3, 'max_depth': 10, 'learning_rate': 0.01, 'gamma': 0, 'colsample_bytree'

# submission

In [7]:
test_predictions = np.zeros(len(test))

for model in models:
    test_predictions += np.maximum(0, np.expm1(model.predict(test))) / len(models)


sample['Premium Amount'] = test_predictions
sample.to_csv('submission1xgb.csv' , index = False)
sample.head()

,id,Premium Amount
0,1200000,808.712921
1,1200001,801.173965
2,1200002,789.772614
3,1200003,788.977814
4,1200004,751.742554
